## Update Product Exploration

### Preparation

In [8]:
# Importing
import json
from os import listdir
from os.path import join
import pandas as pd

# Parameters
recipes_path = r"C:\Users\Fer\Desktop\Code\nourish\data\recipes"

# Secondary functions
def load_json(file_path):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
            return data
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None
    except json.JSONDecodeError:
        print(f"Error decoding JSON from the file: {file_path}")
        return None

### Get Combined Product List

In [9]:
# Parse recipes to collect product list
recipe_list = [f for f in listdir(recipes_path) if "exc_" not in f]
product_list = []

for recipe in recipe_list:
    # Get content
    content = load_json(join(recipes_path, recipe))
    
    # Get ingredients
    for ing in content["ingredients"]:
        product_list.append(ing["ingredient"])

product_list = sorted(list(set(product_list)))
display(product_list)


['Almond Milk',
 'Anise Star',
 'Banana',
 'Black Pepper',
 'Bok Choy',
 'Coconut Milk',
 'Coriander',
 'Cumin',
 'Curry Powder',
 'Frozen Berries',
 'Garlic',
 'Garlic Clove',
 'Ginger',
 'Green Chilli Pepper',
 'Green Onion',
 'Miso Paste',
 'Oat Flakes',
 'Olive Oil',
 'Onion',
 'Peanut Butter',
 'Ramen Noodles',
 'Red Lentils',
 'Salt',
 'Seeds and Nuts Mix',
 'Sesame Oil',
 'Shiitake Mushroom',
 'Soy Sauce',
 'Tomato Paste',
 'Turmeric',
 'Vegetable Broth Cube',
 'Water']

In [10]:
import requests
import json

def call_api_endpoint(url, data=None, headers=None):
    try:
        response = requests.post(url, data=json.dumps(data), headers=headers)
        response.raise_for_status()  # Raise an exception for 4xx or 5xx status codes
        return response.json()  # Parse JSON response
    except requests.exceptions.RequestException as e:
        print("Error:", e)
        return None

def query_product_information(product_query):
    # Example usage:
    api_url = "https://trackapi.nutritionix.com/v2/natural/nutrients"
    api_headers = {
        'Content-Type': 'application/json',
        'x-app-id': '4a82e505',
        'x-app-key': '9946dcfb36a2b92c02754505d0db7fd1'
    }
    api_data = {
        "query": product_query,
        }

    result = call_api_endpoint(api_url, data=api_data, headers=api_headers)
    if result:
        return result
    else:
        print("Failed to fetch data from the API.")

# res = query_product_information("Almond Milk")
# display(res)

In [11]:
import pandas as pd 

def parse_nutrition_response(product, response):

    # Get first appereance and create dataframe
    content = response["foods"][0]

    # Get other nutrients table
    other_nutrients = pd.DataFrame(content["full_nutrients"])

    # Extract data
    # Extract data
    data = {
    # Metadata
    'Product': product,
    'Type': "",
    'ServingQty': content.get("serving_qty", None),
    'ServingUnit': content.get("serving_unit", ""),
    'ServingWeight': content.get("serving_weight_grams", ""),
    'Season': "",
    'Calories': content.get("nf_calories", None),
    # Macro Nutrient Information
    'Protein': content.get("nf_protein", None),
    'Fiber': content.get("nf_dietary_fiber", None),
    'TotalCarboHydrates': content.get("nf_total_carbohydrate", None),
    'TotalFat': content.get("nf_total_fat", None),
    'SaturatedFat': content.get("nf_saturated_fat", None),
    'TransFat': other_nutrients.loc[other_nutrients["attr_id"] == 605, "value"].squeeze() if not other_nutrients.loc[other_nutrients["attr_id"] == 605].empty else None,
    'PolyUnsaturatedFat': other_nutrients.loc[other_nutrients["attr_id"] == 646, "value"].squeeze() if not other_nutrients.loc[other_nutrients["attr_id"] == 646].empty else None,
    'MonoUnsaturatedFat': other_nutrients.loc[other_nutrients["attr_id"] == 645, "value"].squeeze() if not other_nutrients.loc[other_nutrients["attr_id"] == 645].empty else None,
    'AddedSugars': content.get("nf_sugars", None),
    'Cholesterol': content.get("nf_cholesterol", None),
    # Micro Nutrient information
    'Calcium': other_nutrients.loc[other_nutrients["attr_id"] == 301, "value"].squeeze() if not other_nutrients.loc[other_nutrients["attr_id"] == 301].empty else None,
    'Iron': other_nutrients.loc[other_nutrients["attr_id"] == 303, "value"].squeeze() if not other_nutrients.loc[other_nutrients["attr_id"] == 303].empty else None,
    'Magnesium': other_nutrients.loc[other_nutrients["attr_id"] == 304, "value"].squeeze() if not other_nutrients.loc[other_nutrients["attr_id"] == 304].empty else None,
    'Potassium': content.get("nf_potassium", None),
    'Sodium': content.get("nf_sodium", None),
    'Zinc': other_nutrients.loc[other_nutrients["attr_id"] == 309, "value"].squeeze() if not other_nutrients.loc[other_nutrients["attr_id"] == 309].empty else None,
    'Vit A': other_nutrients.loc[other_nutrients["attr_id"] == 320, "value"].squeeze() if not other_nutrients.loc[other_nutrients["attr_id"] == 320].empty else None,
    'Vit B6': other_nutrients.loc[other_nutrients["attr_id"] == 415, "value"].squeeze() if not other_nutrients.loc[other_nutrients["attr_id"] == 415].empty else None,
    'Vit B12': other_nutrients.loc[other_nutrients["attr_id"] == 418, "value"].squeeze() if not other_nutrients.loc[other_nutrients["attr_id"] == 418].empty else None,
    'Vit C': other_nutrients.loc[other_nutrients["attr_id"] == 401, "value"].squeeze() if not other_nutrients.loc[other_nutrients["attr_id"] == 401].empty else None,
    'Vit D': other_nutrients.loc[other_nutrients["attr_id"] == 328, "value"].squeeze() if not other_nutrients.loc[other_nutrients["attr_id"] == 328].empty else None,
    'Vit E': other_nutrients.loc[other_nutrients["attr_id"] == 323, "value"].squeeze() if not other_nutrients.loc[other_nutrients["attr_id"] == 323].empty else None,
    'Vit K': other_nutrients.loc[other_nutrients["attr_id"] == 430, "value"].squeeze() if not other_nutrients.loc[other_nutrients["attr_id"] == 430].empty else None,
}

    product_df = pd.DataFrame(data, index=[0])

    return product_df

# prod_df = parse_nutrition_response("Almond Milk", res)
# display(prod_df)


In [12]:
ingredient_df = pd.DataFrame(content["ingredients"])
product_df = pd.DataFrame()
for ingredient in product_list+["egg"]:
    resp = query_product_information(ingredient)
    df = parse_nutrition_response(ingredient, resp)
    product_df = pd.concat([product_df, df], ignore_index=True)

display(product_df)


C:\Users\Fer\AppData\Local\Temp\ipykernel_6892\1033001229.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  product_df = pd.concat([product_df, df], ignore_index=True)


,Product,Type,ServingQty,ServingUnit,ServingWeight,Season,Calories,Protein,Fiber,TotalCarboHydrates,...,Potassium,Sodium,Zinc,Vit A,Vit B6,Vit B12,Vit C,Vit D,Vit E,Vit K
0,Almond Milk,,1.00,cup,248.80,,56.04,1.06,0.63,8.08,...,36.79,9.59,0.1804,0.000,0.0069,0.0000,0.0000,0.000,1.2815,0.0000
1,Anise Star,,1.00,"tsp, whole",2.10,,7.08,0.37,0.31,1.05,...,30.26,0.34,0.1113,0.336,0.0137,0.0000,0.4410,0.000,NaN,NaN
2,Banana,,1.00,"medium (7"" to 7-7/8"" long)",118.00,,105.02,1.29,3.07,26.95,...,422.44,1.18,0.1770,3.540,0.4331,0.0000,10.2660,0.000,0.1180,0.5900
3,Black Pepper,,1.00,"tsp, ground",2.30,,5.77,0.24,0.58,1.47,...,30.57,0.46,0.0274,0.621,0.0067,0.0000,0.0000,0.000,0.0239,3.7651
4,Bok Choy,,1.00,"cup, shredded",170.00,,20.40,2.65,1.70,3.03,...,630.70,57.80,0.2890,360.400,0.2822,0.0000,44.2000,0.000,0.1530,57.8000
5,Coconut Milk,,1.00,cup,226.00,,445.22,4.57,NaN,6.35,...,497.20,29.38,1.2656,0.000,0.0633,0.0000,2.2600,0.000,NaN,NaN
6,Coriander,,1.00,tsp,0.60,,1.67,0.13,0.06,0.31,...,26.80,1.27,0.0283,1.758,0.0037,0.0000,3.4002,0.000,0.0062,8.1570
7,Cumin,,1.00,"tsp, whole",2.10,,7.88,0.37,0.22,0.93,...,37.55,3.53,0.1008,1.344,0.0091,0.0000,0.1617,0.000,0.0699,0.1134
8,Curry Powder,,1.00,tsp,2.00,,6.50,0.29,1.06,1.12,...,23.40,1.04,0.0940,0.020,0.0021,0.0000,0.0140,0.000,0.5048,1.9960
9,Frozen Berries,,1.00,cup,141.75,,64.72,1.40,5.55,15.41,...,191.38,1.42,0.4324,6.065,0.0648,0.0000,41.5495,0.000,1.0098,17.5035
